# Jupyter Notebook Purpose

- Run all code cells and you will have cleaned all the data and obtain the final dataset

## Group 2 Members

- 1. Melissa Hartwick - [Email](mailto:mhartwic@uwaterloo.ca)
- 2. McKinleigh Needham - [Email](mailto:mjneedha@uwaterloo.ca)
- 3. Daniel Adam Cebula  - [Email](mailto:dacebula@uwaterloo.ca)
- 4. Athithian Selvadurai - [Email](mailto:a6selvad@uwaterloo.ca)
- 5. Aravind Kakarala - [Email](mailto:akakaral@uwaterloo.ca)
- 6. Allan Sales - [Email](mailto:asales@uwaterloo.ca)

In [1]:
import pandas as pd
import numpy as np
import os
import requests  # simple HTTP library for Python
import io        # Tool for working with streams (Input/Ouput data)
import matplotlib.pyplot as plt
import glob

%matplotlib inline

In [2]:
# function to create folders
def create_folder(cwd="DEFAULT", folder_name="DEFAULT"):
    """
    If folder does not exist and it is not an empty string then create the folder and return FilePath.
    If folder does exist and it is not an empty string then return FilePath.
    Otherwise return None.
    """
    import os
    # get file_path for folder_name
    file_path = os.path.join(cwd, folder_name)
    
    # check to see if folder exists and folder name is not empty string ""
    if os.path.exists(file_path) == False and folder_name != "":
        try:
            os.makedirs(file_path) # auto create folder if it does not exist
            # return filepath
            return file_path
        except:
            print("\\n(꒪Д꒪)ノ\tPATH ERROR -- cannot create folder:  ", folder_name)
            return None
    
    # if folder exists then just return filepath
    elif os.path.exists(file_path) == True and folder_name != "":
        return file_path
    
    # for all other conditions just return None
    else:
        return file_path

In [3]:
# Folder Pathways
cwd = os.getcwd()

Toronto_Weather_Directory = create_folder(cwd=cwd, folder_name="TORONTO_WEATHER")
Toronto_Housing_Directory = create_folder(cwd=cwd, folder_name="TORONTO_HOUSING")
TTC_Directory = create_folder(cwd=cwd, folder_name="TTC")
TTC_Subway_Directory = create_folder(cwd=TTC_Directory, folder_name="SUBWAY")
TTC_Bus_Directory = create_folder(cwd=TTC_Directory, folder_name="BUS")
TTC_Streetcar_Directory = create_folder(cwd=TTC_Directory, folder_name="STREETCAR")

# TTC Data Combination

- TTC data will need to be combined into 1 DataFrame

In [4]:
# Get all Filenames for TTC
TTC_Subway_Filenames = []
TTC_Bus_Filenames = []
TTC_Streetcar_Filenames = []

for (dirpath, dirnames, filenames) in os.walk(TTC_Subway_Directory):
    TTC_Subway_Filenames.extend(filenames)
    break

for (dirpath, dirnames, filenames) in os.walk(TTC_Bus_Directory):
    TTC_Bus_Filenames.extend(filenames)
    break
    
for (dirpath, dirnames, filenames) in os.walk(TTC_Streetcar_Directory):
    TTC_Streetcar_Filenames.extend(filenames)
    break
    
len(TTC_Subway_Filenames), len(TTC_Bus_Filenames), len(TTC_Streetcar_Filenames)

(33, 6, 6)

In [5]:
# Generate 1 DataFrame for TTC Subway Data

Subway_Columns = [
    "Date"
    , "Time"
    , "Day"
    , "Station"
    , "Code"
    , "Min Delay"
    , "Min Gap"
    , "Bound"
    , "Line"
    , "Vehicle"
]

df_list = []

for x in TTC_Subway_Filenames:

    df = pd.read_excel(os.path.join(TTC_Subway_Directory, x)
                      , sheet_name=0).iloc[:, :10]
    df.columns = Subway_Columns
        
    df_list.append(df)

# combine the subway dataframes together
df_subway = pd.concat(df_list).reset_index(drop=True)
del df_list, df

# Apply the Code description for the Delay from the SRT and NON-SRT subway lines
df_subway_SRT = df_subway.loc[df_subway["Line"]=="SRT"]
df_subway_NOTSRT = df_subway.loc[df_subway["Line"]!="SRT"]

# Get SRT and NON-SRT codes
Subway_Codes = pd.read_csv(os.path.join(TTC_Directory, "Subway_Codes.csv"))
SRT_Codes = pd.read_csv(os.path.join(TTC_Directory, "SRT_Codes.csv"))

# Merge to get Code Description to the main dataset
df_subway_SRT_codes = pd.merge(df_subway_SRT, SRT_Codes, on=None, left_on="Code", right_on="SRT RMENU CODE", how='left')
df_subway_NOTSRT_codes = pd.merge(df_subway_NOTSRT, Subway_Codes, on=None, left_on="Code", right_on="SUB RMENU CODE", how='left')
df_subway = pd.concat([df_subway_SRT_codes, df_subway_NOTSRT_codes]).reset_index(drop=True)
df_subway = df_subway.rename(columns={"CODE DESCRIPTION": 'CodeDescription'})
df_subway['CodeDescription'] = df_subway['CodeDescription'].fillna(value="NULL")

# convert Date into Date datatype and create a DateTime column too
df_subway["Time"] = df_subway["Time"].astype("str")
df_subway["DateTime"] = df_subway["Date"].astype("str") + " " + df_subway["Time"]
df_subway["DateTime"] = pd.to_datetime(df_subway["DateTime"])
df_subway["DateTimeHour"] = pd.to_datetime(df_subway["DateTime"]).dt.round('H')

# Round the numeric to integer
df_subway["Min Delay"] = df_subway["Min Delay"].round(0).astype(int)
df_subway["Min Gap"] = df_subway["Min Gap"].round(0).astype(int)

# Replace NaN with the string NULL
df_subway['Code'] = df_subway['Code'].fillna(value="NULL")
df_subway['Line'] = df_subway['Line'].fillna(value="NULL")
df_subway['Bound'] = df_subway['Bound'].fillna(value="NULL")

# Combine Bound and Line together
df_subway["LineBound"] = df_subway["Line"] + "-" + df_subway["Bound"]

# Take a subset of the total columns
df_subway = df_subway.loc[:, [
    "DateTimeHour"
    , "DateTime"
    , "Date"
    , "Time"
    , "Station"
    , "LineBound"
    , "Code"
    , "CodeDescription"
    , "Min Delay"
    , "Min Gap"
]]

df_subway.sample(30)

,DateTimeHour,DateTime,Date,Time,Station,LineBound,Code,CodeDescription,Min Delay,Min Gap
57372,2016-08-16 22:00:00,2016-08-16 22:01:00,2016-08-16,22:01,KENNEDY BD STATION,BD-W,MUIR,Injured or ill Customer (On Train) - Medical A...,4,8
81618,2017-12-02 00:00:00,2017-12-02 00:23:00,2017-12-02,00:23,YORKDALE STATION,YU-N,EUSC,Speed Control Equipment,0,0
21843,2014-11-20 00:00:00,2014-11-19 23:40:00,2014-11-19,23:40,BLOOR DANFORTH SUBWAY,BD-NULL,MUGD,Miscellaneous General Delays,0,0
79160,2017-10-17 16:00:00,2017-10-17 15:30:00,2017-10-17,15:30,SHEPPARD WEST STATION,YU-N,TUSC,Operator Overspeeding,0,0
111268,2019-05-31 19:00:00,2019-05-31 19:04:00,2019-05-31,19:04,BROADVIEW STATION,BD-E,MUSC,Miscellaneous Speed Control,0,0
1099,2015-04-26 06:00:00,2015-04-26 05:30:00,2015-04-26,05:30,MCCOWAN STATION TO KEN,SRT-NULL,MRSTM,Scheduled Track Maintenance,0,0
19941,2014-10-15 17:00:00,2014-10-15 17:09:00,2014-10-15,17:09,ST CLAIR STATION,YU-N,MUPAA,Passenger Assistance Alarm Activated - No Trou...,0,0
24534,2015-01-10 11:00:00,2015-01-10 11:22:00,2015-01-10,11:22,ISLINGTON STATION,BD-W,TUSC,Operator Overspeeding,0,0
49695,2016-04-03 22:00:00,2016-04-03 22:00:00,2016-04-03,22:00,YONGE UNIVERSITY LINE,YU/BD-NULL,MUO,Miscellaneous Other,0,0
42208,2015-11-16 17:00:00,2015-11-16 16:36:00,2015-11-16,16:36,CASTLE FRANK STATION,BD-E,SUDP,Disorderly Patron,3,5


In [6]:
df_subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121904 entries, 0 to 121903
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   DateTimeHour     121904 non-null  datetime64[ns]
 1   DateTime         121904 non-null  datetime64[ns]
 2   Date             121904 non-null  datetime64[ns]
 3   Time             121904 non-null  object        
 4   Station          121904 non-null  object        
 5   LineBound        121904 non-null  object        
 6   Code             121904 non-null  object        
 7   CodeDescription  121904 non-null  object        
 8   Min Delay        121904 non-null  int32         
 9   Min Gap          121904 non-null  int32         
dtypes: datetime64[ns](3), int32(2), object(5)
memory usage: 8.4+ MB


In [7]:
# Generate 1 DataFrame for Bus Data

Bus_Columns = [
    "Report Date"
    , "Route"
    , "Time"
    , "Day"
    , "Location"
    , "Incident"
    , "Min Delay"
    , "Min Gap"
    , "Direction"
    , "Vehicle"
]

df_list = []

for x in TTC_Bus_Filenames:
    
    xlsx = pd.ExcelFile(os.path.join(TTC_Bus_Directory, x))
    
    for y in xlsx.sheet_names:
        
        if y == "Apr 2019":
            df = pd.read_excel(xlsx, sheet_name=y).iloc[:, :11]
            df = df.drop(["Incident ID"], axis=1)
            df.columns = Bus_Columns
        else:
            df = pd.read_excel(xlsx, sheet_name=y).iloc[:, :10]
            df.columns = Bus_Columns

        df_list.append(df)

# combine the bus dataframes together
df_bus = pd.concat(df_list).reset_index(drop=True)
del df_list, df

# convert Date into Date datatype and create a DateTime column too
df_bus = df_bus.rename(columns={"Report Date": 'Date'})
df_bus["Time"] = df_bus["Time"].astype("str").apply(lambda x:x[-8:])
df_bus["DateTime"] = pd.to_datetime(df_bus["Date"].astype("str") + " " + df_bus["Time"])
df_bus["DateTimeHour"] = df_bus["DateTime"].dt.round('H')

# Round the numeric to integer
df_bus = df_bus.dropna(subset=['Min Delay', 'Min Gap'])
df_bus["Min Delay"] = df_bus["Min Delay"].round(0).astype(int)
df_bus["Min Gap"] = df_bus["Min Gap"].round(0).astype(int)

# Replace NaN with the string NULL
df_bus['Location'] = df_bus['Location'].fillna(value="NULL")
df_bus['Incident'] = df_bus['Incident'].fillna(value="NULL")
df_bus['Direction'] = df_bus['Direction'].fillna(value="NULL")
df_bus['Vehicle'] = df_bus['Vehicle'].fillna(value="NULL")

# Drop rows where delay or gap is unknown
df_bus = df_bus.dropna(subset=['Min Delay', 'Min Gap'])

# Take a subset of the total columns
df_bus = df_bus.loc[:, [
    "DateTimeHour"
    , "DateTime"
    , "Date"
    , "Time"
    , "Route"
    , "Location"
    , "Incident"
    , "Min Delay"
    , "Min Gap"
]]

df_bus.sample(30)

,DateTimeHour,DateTime,Date,Time,Route,Location,Incident,Min Delay,Min Gap
76528,2014-10-19 09:00:00,2014-10-19 08:42:00,2014-10-19,08:42:00,165,Yorkmills station,Late Leaving Garage,42,84
216362,2016-08-15 06:00:00,2016-08-15 05:47:00,2016-08-15,05:47:00,57,Midland and Steeles,Late Leaving Garage,17,30
122284,2015-05-04 11:00:00,2015-05-04 11:00:00,2015-05-04,11:00:00,106,Downsview Stn,Mechanical,10,20
155488,2015-10-15 17:00:00,2015-10-15 16:31:00,2015-10-15,16:31:00,52,LAWRENCE WEST STN,Mechanical,11,22
307208,2017-11-11 17:00:00,2017-11-11 17:00:00,2017-11-11,17:00:00,17,Comstock on Birchmount,General Delay,15,0
261008,2017-03-17 17:00:00,2017-03-17 16:40:00,2017-03-17,16:40:00,52,Lawrence and Bathrust,Utilized Off Route,2,5
11267,2014-02-05 20:00:00,2014-02-05 20:06:00,2014-02-05,20:06:00,199,Finch Ave and Brimley,Mechanical,7,14
254849,2017-02-10 15:00:00,2017-02-10 15:18:00,2017-02-10,15:18:00,12,Kingston and Queensberry,Mechanical,13,27
332690,2018-03-12 08:00:00,2018-03-12 08:00:00,2018-03-12,08:00:00,106,sentinel and Pond,Diversion,360,370
232413,2016-10-25 15:00:00,2016-10-25 14:58:00,2016-10-25,14:58:00,24,Victoria Park Station,Late Leaving Garage,8,16


In [8]:
df_bus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 453472 entries, 0 to 454420
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   DateTimeHour  453472 non-null  datetime64[ns]
 1   DateTime      453472 non-null  datetime64[ns]
 2   Date          453472 non-null  datetime64[ns]
 3   Time          453472 non-null  object        
 4   Route         453472 non-null  int64         
 5   Location      453472 non-null  object        
 6   Incident      453472 non-null  object        
 7   Min Delay     453472 non-null  int32         
 8   Min Gap       453472 non-null  int32         
dtypes: datetime64[ns](3), int32(2), int64(1), object(3)
memory usage: 31.1+ MB


In [9]:
# Generate 1 DataFrame for Streetcar Data

Streetcar_Columns = [
    "Report Date"
    , "Route"
    , "Time"
    , "Day"
    , "Location"
    , "Incident"
    , "Min Delay"
    , "Min Gap"
    , "Direction"
    , "Vehicle"
]

df_list = []

for x in TTC_Streetcar_Filenames:
    
    xlsx = pd.ExcelFile(os.path.join(TTC_Streetcar_Directory, x))
    
    for y in xlsx.sheet_names:
        
        if y == "Apr 2019":
            df = pd.read_excel(xlsx, sheet_name=y).iloc[:, :11]
            df = df.drop(["Incident ID"], axis=1)
            df.columns = Streetcar_Columns
        else:
            df = pd.read_excel(xlsx, sheet_name=y).iloc[:, :10]
            df.columns = Streetcar_Columns

        df_list.append(df)

# combine the bus dataframes together
df_streetcar = pd.concat(df_list).reset_index(drop=True)
del df_list, df

# convert Date into Date datatype and create a DateTime column too
df_streetcar = df_streetcar.rename(columns={"Report Date": 'Date'})
df_streetcar["Time"] = df_streetcar["Time"].astype("str").apply(lambda x:x[-8:])
df_streetcar["DateTime"] = pd.to_datetime(df_streetcar["Date"].astype("str") + " " + df_streetcar["Time"])
df_streetcar["DateTimeHour"] = df_streetcar["DateTime"].dt.round('H')

# Round the numeric to integer
df_streetcar = df_streetcar.dropna(subset=['Min Delay', 'Min Gap'])
df_streetcar["Min Delay"] = df_streetcar["Min Delay"].round(0).astype(int)
df_streetcar["Min Gap"] = df_streetcar["Min Gap"].round(0).astype(int)

# Replace NaN with the string NULL
df_streetcar['Location'] = df_streetcar['Location'].fillna(value="NULL")
df_streetcar['Incident'] = df_streetcar['Incident'].fillna(value="NULL")
df_streetcar['Direction'] = df_streetcar['Direction'].fillna(value="NULL")
df_streetcar['Vehicle'] = df_streetcar['Vehicle'].fillna(value="NULL")

# Drop rows where delay or gap is unknown
df_streetcar = df_streetcar.dropna(subset=['Min Delay', 'Min Gap'])

# Take a subset of the total columns
df_streetcar = df_streetcar.loc[:, [
    "DateTimeHour"
    , "DateTime"
    , "Date"
    , "Time"
    , "Route"
    , "Location"
    , "Incident"
    , "Min Delay"
    , "Min Gap"
]]

df_streetcar.sample(30)

,DateTimeHour,DateTime,Date,Time,Route,Location,Incident,Min Delay,Min Gap
56442,2018-05-02 07:00:00,2018-05-02 07:08:00,2018-05-02,07:08:00,504,Leslie Barns,Late Leaving Garage,4,8
15594,2015-04-20 07:00:00,2015-04-20 06:58:00,2015-04-20,06:58:00,504,Broadview and Queen,Late Leaving Garage,2,4
19101,2015-08-15 02:00:00,2015-08-15 01:30:00,2015-08-15,01:30:00,306,Gerrard at Carlaw,Mechanical,38,48
16410,2015-05-19 15:00:00,2015-05-19 15:18:00,2015-05-19,15:18:00,504,Queen and Roncesvalles,Mechanical,4,8
29111,2016-06-04 20:00:00,2016-06-04 20:00:00,2016-06-04,20:00:00,506,Main Station to High Park,General Delay,8,16
42693,2017-05-23 09:00:00,2017-05-23 08:43:00,2017-05-23,08:43:00,506,Parliament and Gerrard.,Mechanical,60,65
43306,2017-06-07 06:00:00,2017-06-07 05:36:00,2017-06-07,05:36:00,512,Roncy Yard,Late Leaving Garage,25,4
57272,2018-05-18 06:00:00,2018-05-18 05:49:00,2018-05-18,05:49:00,511,Bathurst and King,Mechanical,5,10
24335,2016-01-29 06:00:00,2016-01-29 06:03:00,2016-01-29,06:03:00,504,Roncesvalles Carhouse.,Mechanical,3,6
22474,2015-11-30 06:00:00,2015-11-30 06:30:00,2015-11-30,06:30:00,501,Roncevalles and Queen,Diversion,5,9


In [10]:
df_streetcar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78393 entries, 0 to 78524
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   DateTimeHour  78393 non-null  datetime64[ns]
 1   DateTime      78393 non-null  datetime64[ns]
 2   Date          78393 non-null  datetime64[ns]
 3   Time          78393 non-null  object        
 4   Route         78393 non-null  int64         
 5   Location      78393 non-null  object        
 6   Incident      78393 non-null  object        
 7   Min Delay     78393 non-null  int32         
 8   Min Gap       78393 non-null  int32         
dtypes: datetime64[ns](3), int32(2), int64(1), object(3)
memory usage: 5.4+ MB


In [11]:
# Write the 3 dataframes to files
df_subway.to_csv(os.path.join(TTC_Directory, "2014-2019-Subway_Data.csv"), index=False)
df_bus.to_csv(os.path.join(TTC_Directory, "2014-2019-Bus_Data.csv"), index=False)
df_streetcar.to_csv(os.path.join(TTC_Directory, "2014-2019-Streetcar_Data.csv"), index=False)
del df_subway, df_bus, df_streetcar

# Combine TTC data with Canadian Historical Weather

- do so on data summarized from 3 historical toronto weather stations that reported from 2014 - 2019

In [12]:
# connect to TTC Subway, Bus and Streetcar Delay Data and Toronto Historical Weather
df_subway = pd.read_csv(os.path.join(TTC_Directory, "2014-2019-Subway_Data.csv")
                       , parse_dates=["DateTimeHour", "Date"])
df_bus = pd.read_csv(os.path.join(TTC_Directory, "2014-2019-Bus_Data.csv")
                       , parse_dates=["DateTimeHour", "Date"])
df_streetcar = pd.read_csv(os.path.join(TTC_Directory, "2014-2019-Streetcar_Data.csv")
                       , parse_dates=["DateTimeHour", "Date"])
df_weather = pd.read_csv(os.path.join(Toronto_Weather_Directory, "Toronto_Weather.csv")
                       , parse_dates=["Date/Time (LST)"])
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52584 entries, 0 to 52583
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date/Time (LST)      52584 non-null  datetime64[ns]
 1   Temp (°C)            52582 non-null  float64       
 2   Precip. Amount (mm)  52573 non-null  float64       
 3   Wind Dir (10s deg)   52567 non-null  float64       
 4   Wind Spd (km/h)      52575 non-null  float64       
 5   Stn Press (kPa)      52582 non-null  float64       
 6   Hmdx                 9535 non-null   float64       
 7   Wind Chill           12403 non-null  float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 3.2 MB


In [13]:
# Merge the data on the date and hour to match weather
df_subway_weather = pd.merge(df_subway, df_weather, on=None, left_on="DateTimeHour", right_on="Date/Time (LST)", how='inner')
df_bus_weather = pd.merge(df_bus, df_weather, on=None, left_on="DateTimeHour", right_on="Date/Time (LST)", how='inner')
df_streetcar_weather = pd.merge(df_streetcar, df_weather, on=None, left_on="DateTimeHour", right_on="Date/Time (LST)", how='inner')

In [14]:
# Get Holidays
Holiday_Directory = create_folder(cwd=cwd, folder_name="HOLIDAY")

URL = "https://raw.githubusercontent.com/uWaterloo/Datasets/master/Holidays/holidays.csv"

with requests.get(URL, stream=True) as response:
    # location where it will be saved
    filepath = os.path.join(Holiday_Directory, f"Holidays.csv")
    # save the .xlsx
    with open(filepath, "wb") as file:
        for chunk in response.iter_content(chunk_size=128):
            file.write(chunk)
            
df_holiday = pd.read_csv(os.path.join(Holiday_Directory, "Holidays.csv"), parse_dates=["date"])
df_holiday = df_holiday.rename(columns={"holiday": 'HolidayName'})

df_subway_weather = pd.merge(df_subway_weather, df_holiday, on=None, left_on="Date", right_on="date", how='left')
df_bus_weather = pd.merge(df_bus_weather, df_holiday, on=None, left_on="Date", right_on="date", how='left')
df_streetcar_weather = pd.merge(df_streetcar_weather, df_holiday, on=None, left_on="Date", right_on="date", how='left')

df_subway_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121903 entries, 0 to 121902
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   DateTimeHour         121903 non-null  datetime64[ns]
 1   DateTime             121903 non-null  object        
 2   Date                 121903 non-null  datetime64[ns]
 3   Time                 121903 non-null  object        
 4   Station              121903 non-null  object        
 5   LineBound            121903 non-null  object        
 6   Code                 121902 non-null  object        
 7   CodeDescription      119224 non-null  object        
 8   Min Delay            121903 non-null  int64         
 9   Min Gap              121903 non-null  int64         
 10  Date/Time (LST)      121903 non-null  datetime64[ns]
 11  Temp (°C)            121899 non-null  float64       
 12  Precip. Amount (mm)  121869 non-null  float64       
 13  Wind Dir (10s 

In [15]:
# fill NaN with "Not A Holiday"
df_subway_weather['HolidayName'] = df_subway_weather['HolidayName'].fillna(value="Not A Holiday")
df_bus_weather['HolidayName'] = df_bus_weather['HolidayName'].fillna(value="Not A Holiday")
df_streetcar_weather['HolidayName'] = df_streetcar_weather['HolidayName'].fillna(value="Not A Holiday")

df_subway_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121903 entries, 0 to 121902
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   DateTimeHour         121903 non-null  datetime64[ns]
 1   DateTime             121903 non-null  object        
 2   Date                 121903 non-null  datetime64[ns]
 3   Time                 121903 non-null  object        
 4   Station              121903 non-null  object        
 5   LineBound            121903 non-null  object        
 6   Code                 121902 non-null  object        
 7   CodeDescription      119224 non-null  object        
 8   Min Delay            121903 non-null  int64         
 9   Min Gap              121903 non-null  int64         
 10  Date/Time (LST)      121903 non-null  datetime64[ns]
 11  Temp (°C)            121899 non-null  float64       
 12  Precip. Amount (mm)  121869 non-null  float64       
 13  Wind Dir (10s 

In [16]:
# Add relevant Date values for analysis
df_subway_weather["DayName"] = df_subway_weather["DateTimeHour"].dt.day_name()
df_subway_weather["Hour"] = df_subway_weather["DateTimeHour"].dt.hour
df_subway_weather["WeekOfYear"] = df_subway_weather["DateTimeHour"].dt.isocalendar().week
df_subway_weather["MonthName"] = df_subway_weather["DateTimeHour"].dt.month_name()
df_subway_weather["BusinessQuarter"] = df_subway_weather["DateTimeHour"].dt.quarter

df_bus_weather["DayName"] = df_bus_weather["DateTimeHour"].dt.day_name()
df_bus_weather["Hour"] = df_bus_weather["DateTimeHour"].dt.hour
df_bus_weather["WeekOfYear"] = df_bus_weather["DateTimeHour"].dt.isocalendar().week
df_bus_weather["MonthName"] = df_bus_weather["DateTimeHour"].dt.month_name()
df_bus_weather["BusinessQuarter"] = df_bus_weather["DateTimeHour"].dt.quarter

df_streetcar_weather["DayName"] = df_streetcar_weather["DateTimeHour"].dt.day_name()
df_streetcar_weather["Hour"] = df_streetcar_weather["DateTimeHour"].dt.hour
df_streetcar_weather["WeekOfYear"] = df_streetcar_weather["DateTimeHour"].dt.isocalendar().week
df_streetcar_weather["MonthName"] = df_streetcar_weather["DateTimeHour"].dt.month_name()
df_streetcar_weather["BusinessQuarter"] = df_streetcar_weather["DateTimeHour"].dt.quarter

# take a subset of the final merge columns
df_subway_weather = df_subway_weather.loc[:, [
    "DateTime"
    , "Date"
    , "BusinessQuarter"
    , "MonthName"
    , "WeekOfYear"
    , "DayName"
    , "HolidayName"
    , "Time"
    , "Station"
    , "LineBound"
    , "Code"
    , "CodeDescription"
    , "Temp (°C)"
    , "Precip. Amount (mm)"
    , "Wind Dir (10s deg)"
    , "Wind Spd (km/h)"
    , "Stn Press (kPa)"
    , "Min Delay"
    , "Min Gap"
]]

# take a subset of the final merge columns
df_bus_weather = df_bus_weather.loc[:, [
    "DateTime"
    , "Date"
    , "BusinessQuarter"
    , "MonthName"
    , "WeekOfYear"
    , "DayName"
    , "HolidayName"
    , "Time"
    , "Route"
    , "Location"
    , "Incident"
    , "Temp (°C)"
    , "Precip. Amount (mm)"
    , "Wind Dir (10s deg)"
    , "Wind Spd (km/h)"
    , "Stn Press (kPa)"
    , "Min Delay"
    , "Min Gap"
]]

# take a subset of the final merge columns
df_streetcar_weather = df_streetcar_weather.loc[:, [
    "DateTime"
    , "Date"
    , "BusinessQuarter"
    , "MonthName"
    , "WeekOfYear"
    , "DayName"
    , "HolidayName"
    , "Time"
    , "Route"
    , "Location"
    , "Incident"
    , "Temp (°C)"
    , "Precip. Amount (mm)"
    , "Wind Dir (10s deg)"
    , "Wind Spd (km/h)"
    , "Stn Press (kPa)"
    , "Min Delay"
    , "Min Gap"
]]

In [17]:
# Write these final files to folder

Final_Data_Directory = create_folder(cwd=cwd, folder_name="FINAL_DATA")

# Write the 3 dataframes to files
df_subway_weather.to_csv(os.path.join(Final_Data_Directory, "2014-2019-Subway_Weather_Data.csv"), index=False)
df_bus_weather.to_csv(os.path.join(Final_Data_Directory, "2014-2019-Bus_Weather_Data.csv"), index=False)
df_streetcar_weather.to_csv(os.path.join(Final_Data_Directory, "2014-2019-Streetcar_Weather_Data.csv"), index=False)
del df_subway_weather, df_bus_weather, df_streetcar_weather